# Measuring quality

In [1]:
import math
from collections import namedtuple
import typing as tp
import numpy as np

### Precision@K



In [2]:
def precicionAtK(actual: tp.List[int], predicted: tp.List[float], k: int) -> float:
    if k == 0:
        return 0.0
    
    pos_label = max(actual)
    
    if pos_label == 0:
        return 0.0

    predicted = np.argsort(predicted)[::-1]
    actual = np.take(actual, predicted[:k])
    number_of_relevant = np.sum(actual == pos_label)
    return number_of_relevant / k

In [3]:
PrecisionTestCase = namedtuple('precision_at_k', ['actual', 'predicted', 'k', 'ans'])

In [4]:
precision_tests = [
                   PrecisionTestCase([0,1,1], [0.67, 0.88, 0.66], 3, 0.66),
                   PrecisionTestCase([0,1,1], [0.67, 0.88, 0.66], 2, 0.5),
                   PrecisionTestCase([0,1,1], [0.67, 0.88, 0.66], 1, 1.0),
                   PrecisionTestCase([0,0,0], [0.67, 0.88, 0.66], 3, 0.0),
                   PrecisionTestCase([0,1,1], [0.88, 0.67, 0.66], 1, 0.0),
                   PrecisionTestCase([0,1,1,1], [0.67, 0.88, 0.66, 0.6], 3, 0.66),
                  ]

In [5]:
for case in precision_tests:
    result = precicionAtK(case.actual, case.predicted, case.k)
    assert math.isclose(case.ans, result, rel_tol=0.01), f"{result}!= {case.ans}"

###  Average Precision@K

![image-3.png](attachment:image-3.png)

In [6]:
def averagePrecisionAtK(y_true: tp.List[int], y_score: tp.List[float], k: int) -> float:
    pos_label = max(y_true)

    if pos_label == 0:
        return 0.0

    n_pos = y_true.count(pos_label)

    order = np.argsort(y_score)[::-1][:k]
    y_true = np.asarray(y_true)[order]
    score = 0
    for i in range(len(y_true)):
        if y_true[i] == pos_label:
            prec = 0
            for j in range(0, i + 1):
                if y_true[j] == pos_label:
                    prec += 1.0
            prec /= (i + 1.0)
            score += prec

    if n_pos == 0:
        return 0

    return score / n_pos

In [7]:
ApkTestCase = namedtuple('Avgprecision_at_k', ['actual', 'predicted', 'k', 'ans'])

avg_precision_at_k_test_cases = [
                                 ApkTestCase([0,1,1], [0.88, 0.67, 0.66], 3, 0.58),
                                 ApkTestCase([0,0,0], [0.88, 0.67, 0.66], 3, 0.0),
                                 ApkTestCase([1,0,0], [0.88, 0.67, 0.66], 3, 1.0),
                                 ApkTestCase([1,0,0], [0.88, 0.67, 0.66], 1, 1.0),
                                 ApkTestCase([1,0,1], [0.3, 0.7, 0.6], 3, 0.58)
                                ]

for case in avg_precision_at_k_test_cases:
    result = averagePrecisionAtK(case.actual, case.predicted, case.k)
    assert math.isclose(case.ans, result, rel_tol=0.01), f"{result}!= {case.ans}"

# Mean Average Precision@K



In [8]:
def meanAveragePrecisionAtK(actual: tp.List[tp.List[int]], predicted: tp.List[tp.List[float]], k: int) -> float:
    return np.mean([averagePrecisionAtK(a, p, k) for a, p in zip(actual, predicted)])

#  Discounted Cumulative Gain


In [10]:
def discountedCumulativeGainAtK(actual: tp.List[int], predicted: tp.List[float], k: int) -> float:
    order = np.argsort(predicted)[::-1]
    actual = np.take(actual, order[:k])
    discounts = np.log2(np.arange(len(actual)) + 2)
    return np.sum(actual / discounts)

In [11]:
DCGTestCase = namedtuple('DCG_at_k', ['actual', 'predicted', 'k', 'ans'])

dcg_test_cases = [
                DCGTestCase([3,2,1,0], [2,3,1,0], 3, 4.39),
                DCGTestCase([3,2,1,0], [3,2,1,0], 3, 4.76),
                DCGTestCase([3,2,1,0], [0,1,2,3], 3, 1.63),
                DCGTestCase([3,2,1,0], [0,1,2,3], 2, 0.63),
]

for case in dcg_test_cases:
    result = discountedCumulativeGainAtK(case.actual, case.predicted, case.k)
    assert math.isclose(case.ans, result, rel_tol=0.01), f"{result}!= {case.ans}"

### follow up - parametrizedDCG


In [12]:
def parametrizedDCG(actual: tp.List[int], predicted: tp.List[float], k: int=1, gain_type: str='linear') -> float:
    order = np.argsort(predicted)[::-1]
    actual = np.take(actual, order[:k])

    if gain_type == 'nonlinear':
        gains = np.power(2, actual)-1
    else:
        gains = actual

    discounts = np.log2(np.arange(len(actual)) + 2)
    return np.sum(gains / discounts)

In [13]:
ParamDCGTestCase = namedtuple('ParamDCG_at_k', ['actual', 'predicted', 'k', 'gain_type' ,'ans'])

param_dcg_test_cases = [
                ParamDCGTestCase([3,2,1,0], [2,3,1,0], 3,'linear', 4.39),
                ParamDCGTestCase([3,2,1,0], [2,3,1,0], 3,'nonlinear', 7.91),
]

for case in param_dcg_test_cases:
    result = parametrizedDCG(case.actual, case.predicted, case.k, case.gain_type)
    assert math.isclose(case.ans, result, rel_tol=0.01), f"{result}!= {case.ans}"

### Normalized Discounted Cumulative Gain


In [14]:
def normalizedDiscountedCumulativeGainAtK(actual: tp.List[int], predicted: tp.List[float], k: int) -> float:
    predicted_items_dcg = discountedCumulativeGainAtK(actual, predicted, k)
    ground_truth_dcg = discountedCumulativeGainAtK(actual, actual, k)
    return predicted_items_dcg / ground_truth_dcg

In [15]:
actual = [3,2,1,0]
predicted = [2,3,1,0]
k = 3
result = normalizedDiscountedCumulativeGainAtK(actual, predicted, k)
ans = 0.92
assert math.isclose(ans, result, rel_tol=0.01), f"{result}!= {ans}"

# Recriprocal Rank


In [16]:
def ReciprocalRank(actual: tp.List[int], predicted: tp.List[float]) -> float:
    pos_label = max(actual)
    order = np.argsort(predicted)[::-1]
    actual = np.take(actual, order)
    
    if max(actual) == 0:
        return 0.0
    
    first_relevant_pos = list(actual).index(pos_label) + 1
    return 1 / first_relevant_pos

In [17]:
ReciprocalRankCase = namedtuple('ReciprocalRank', ['actual', 'predicted', 'ans'])

reciprocal_rank_test_cases = [
                ReciprocalRankCase([0,1,0], [0.75, 0.73, 0.72], 0.5),
                ReciprocalRankCase([0,0,0], [0.75, 0.73, 0.72], 0.0),
                ReciprocalRankCase([1,1,1], [0.75, 0.73, 0.72], 1.0),
                ReciprocalRankCase([1,1,1], [0.73, 0.75, 0.72], 1.0),
                ReciprocalRankCase([1,0,0], [0.73, 0.75, 0.72], 0.5),
]

for case in reciprocal_rank_test_cases:
    result = ReciprocalRank(case.actual, case.predicted)
    assert math.isclose(case.ans, result, rel_tol=0.01), f"{result}!= {case.ans}"

### Materials

https://github.com/statisticianinstilettos/recmetrics